# fonction de creation de tableau de taux d'echange


In [5]:
import requests
import pandas as pd
from io import BytesIO
import zipfile
import os
from datetime import datetime

def download_and_process_csv(url, list_currency=None, start_date=None, end_date=None, folder_path=None):
    # Télécharger le fichier ZIP depuis l'URL
    today = datetime.now().strftime('%Y%m%d')
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content), 'r') as zip_ref:
        # Extraire le fichier CSV contenu dans le ZIP
        csv_filename = zip_ref.namelist()[0]
        extraction_path = 'dossier_taux_bce'
        zip_ref.extractall(extraction_path)
        with zip_ref.open(csv_filename) as csv_file:
            # Charger le CSV en DataFrame pandas
            df = pd.read_csv(csv_file)
    df.rename(columns={'Date': 'date'}, inplace=True)
    
    df['date'] = pd.to_datetime(df['date'])
    
    df.drop("Unnamed: 42",axis=1,inplace=True)

    # Créer un nouveau DataFrame avec les colonnes de list_currency
    if list_currency:
        currency_columns = ['date'] + list_currency
        df = df[currency_columns]
    
    # Filtrer le DataFrame en fonction de start_date et end_date
    if start_date:
        if end_date:
            mask = (df['date'] >= start_date) & (df['date'] <= end_date)
            df = df.loc[mask]
        
        else:
            mask = df['date'] >= start_date
            df = df.loc[mask]

    else:
        if end_date: 
            mask = df['date'] <= end_date
            df = df.loc[mask]
   

    melted_df = df.melt(id_vars=['date'], var_name='key_change', value_name='taux').sort_values([ 'date','key_change'],ascending=False)
    melted_df['x_vers_euro'] = 1 / melted_df['taux']
    
    # Exporter le DataFrame en tant que CSV dans le dossier spécifié
    if folder_path:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        output_filename = os.path.join(folder_path, f'taux_bce_{today}.csv')
        melted_df.to_csv(output_filename, index=False)
        print(f"Fichier CSV exporté avec succès vers {output_filename}")
    else:
        folder_path=extraction_path
        output_filename = os.path.join(folder_path, f'taux_bce_{today}.csv')
        melted_df.to_csv(output_filename, index=False)
        print(f"Fichier CSV exporté avec succès vers {output_filename}")




In [6]:
# Exemple d'utilisation de la fonction
url ='https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist.zip?bd049ef55462a7bf91e6643cd0735a01'
# list_currency = ['USD','JPY','CNY','GBP']
# # #list_currency = None
# # start_date = None
# # end_date = None
# # folder_path = None
# start_date = pd.to_datetime('2023-01-01')
# end_date = pd.to_datetime('2023-06-30')

# #download_and_process_csv(url, list_currency, start_date, end_date, folder_path)
download_and_process_csv(url)

Fichier CSV exporté avec succès vers dossier_taux_bce/taux_bce_20230817.csv


In [ ]:
import pandas as pd
from datetime import datetime

# Charger le fichier CSV dans un DataFrame
file_path = "dossier_taux_bce/taux_bce_20230817.csv"
df = pd.read_csv(file_path)

# # Conversion de la colonne "key_change" en format chaîne si elle ne l'est pas déjà
# df['key_change'] = df['key_change'].astype(str)

# Fonction pour convertir une date en format SAS
def convert_to_sas_date(date):
    sas_date = (date - datetime(1960, 1, 1)).days
    return sas_date

# Ajouter la nouvelle colonne "key_change" au DataFrame
df['key_change'] = df['key_change'] + df['date'].apply(lambda x: convert_to_sas_date(pd.to_datetime(x)))

# Enregistrer le DataFrame avec la nouvelle colonne dans un nouveau fichier CSV
new_file_path = "chemin/vers/votre/nouveau_fichier.csv"
df.to_csv(new_file_path, index=False)

print("Nouveau fichier CSV créé avec la colonne 'key_change'.")


In [9]:
import pandas as pd
from datetime import datetime

# Charger le fichier CSV dans un DataFrame
file_path = "dossier_taux_bce/taux_bce_20230817.csv"
df = pd.read_csv(file_path)

df['date'] = pd.to_datetime(df['date'])




In [10]:
df['date_sas'] = (df['date'] - pd.Timestamp('1960-01-01')) // pd.Timedelta('1s')

In [11]:
df

,date,key_change,taux,x_vers_euro,date_sas
0,2023-08-17,ZAR,20.7417,0.048212,2007849600
1,2023-08-17,USD,1.0900,0.917431,2007849600
2,2023-08-17,TRY,29.5376,0.033855,2007849600
3,2023-08-17,TRL,NaN,NaN,2007849600
4,2023-08-17,THB,38.5420,0.025946,2007849600
...,...,...,...,...,...
258623,1999-01-04,CHF,1.6168,0.618506,1231027200
258624,1999-01-04,CAD,1.8004,0.555432,1231027200
258625,1999-01-04,BRL,NaN,NaN,1231027200
258626,1999-01-04,BGN,NaN,NaN,1231027200


In [12]:
mask = df['date']=='24/04/2023'

In [13]:
df[mask]

,date,key_change,taux,x_vers_euro,date_sas
3362,2023-04-24,ZAR,19.94760,0.050131,1997913600
3363,2023-04-24,USD,1.10020,0.908926,1997913600
3364,2023-04-24,TRY,21.35070,0.046837,1997913600
3365,2023-04-24,TRL,NaN,NaN,1997913600
3366,2023-04-24,THB,37.85800,0.026414,1997913600
3367,2023-04-24,SKK,NaN,NaN,1997913600
3368,2023-04-24,SIT,NaN,NaN,1997913600
3369,2023-04-24,SGD,1.46840,0.681013,1997913600
3370,2023-04-24,SEK,11.30900,0.088425,1997913600
3371,2023-04-24,RUB,NaN,NaN,1997913600


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258628 entries, 0 to 258627
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         258628 non-null  datetime64[ns]
 1   key_change   258628 non-null  object        
 2   taux         197375 non-null  float64       
 3   x_vers_euro  197375 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 7.9+ MB


In [7]:
df['key_change1'] = df['key_change']+'_'+ df['date'].apply(lambda x: convert_to_sas_date(x))
df

TypeError: can only concatenate str (not "int") to str

In [ ]:
import pandas as pd

# Exemple de fonction pour combiner les valeurs de deux colonnes
def ma_fonction(val1, val2):
    # Ici, vous pouvez définir comment combiner val1 et val2 pour obtenir la nouvelle valeur
    nouvelle_valeur = val1 + val2
    return nouvelle_valeur

# Création d'un DataFrame d'exemple
data = {'colonne1': [1, 2, 3],
        'colonne2': [4, 5, 6]}

df = pd.DataFrame(data)

# Application de la fonction pour créer une nouvelle colonne
df['nouvelle_colonne'] = df.apply(lambda row: ma_fonction(row['colonne1'], row['colonne2']), axis=1)

# Affichage du DataFrame résultant
print(df)
